# GWP # 1

s101 MScFE 560 Financial Markets 

Jaepil Choi

## My Portfolio: Portfolio A

Buy 1 stock and short 1 stock

In [3]:
from typing import Tuple

import pandas as pd
import numpy as np

from itertools import permutations

## Step 1

For each investment, you are given statistics in this .csv file the stock’s.
You also have the correlation and covariance matrix of historical returns (download this
.csv file)

1. Average return
2. Volatility
3. Skewness
4. Kurtosis

You must compute the portfolio’s average return and portfolio volatility. You do not
need to compute the portfolio skewness and portfolio kurtosis.

In [4]:
statistics_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', nrows=4, index_col=0)
statistics_df = statistics_df.iloc[:, :5]
statistics_df

,AAPL,AMZN,NFLX,META,GOOG
Mean,0.044858,0.010362,-0.029675,-0.006934,0.029035
St Dev,0.009320,0.009557,0.013409,0.011582,0.008376
Skew,-0.335173,-0.312865,-3.101654,-1.936506,-0.197700
Kurt,5.034093,4.729806,41.968015,23.289236,4.310171


In covariance matrix, `GOOGL` (Alphabet Inc Class A) is given instead of `GOOG` (Alphabet Inc Class C). 

To match the name, I changed `GOOGL` to `GOOG`

In [5]:
corr_matrix_df = pd.read_csv('MScFE560_FM_GWP1_Data.csv', skiprows=5, nrows=6, header=1, index_col=0)
corr_matrix_df = corr_matrix_df.iloc[:, :5]
corr_matrix_df.rename({'GOOGL': 'GOOG'}, axis=0, inplace=True)
corr_matrix_df.rename({'GOOGL': 'GOOG'}, axis=1, inplace=True)
corr_matrix_df

,AAPL,AMZN,NFLX,META,GOOG
AAPL,1.000000,0.660739,0.460041,0.594785,0.698431
AMZN,0.660739,1.000000,0.593812,0.626198,0.679323
NFLX,0.460041,0.593812,1.000000,0.515892,0.492697
META,0.594785,0.626198,0.515892,1.000000,0.668024
GOOG,0.698431,0.679323,0.492697,0.668024,1.000000


### 1-1 Portfolio's average return

In [6]:
mean = statistics_df.loc['Mean', :] 
std = statistics_df.loc['St Dev', :]

In [7]:
sid_list = statistics_df.columns
sid_list

Index(['AAPL', 'AMZN', 'NFLX', 'META', 'GOOG'], dtype='object')

Since I can long 1 stock and short 1 stock, there are $ 2^5 $ number of cases

In [8]:
longshort_cases = permutations(sid_list, 2)
longshort_cases = list(longshort_cases)

Portfolio return formula:

$$ PortfolioReturn = w_a * r_a + w_b * r_b $$

What is "weight"?

Weight is the position betted on the stock. 

What does it mean to have negative weight? 

It means you're shorting. 

By shorting, you borrow stocks and sell them to the market: which gives you cash. 

You can use this cash to long the stock. 

As you can see, long-short can have varing book size. 

In this assignment, I'm longing 1 AAPL and shorting 1 NFLX. 

As of 2022-10-24, AAPL is \$147.27 and NFLX is \$289.57

That is, weights should be:

`AAPL:NFLX = +147.27:-289.57`


In [18]:
A = 147.27
B = -289.57

gross_position = abs(A) + abs(B)
weight_A = A / gross_position
weight_B = B / gross_position

WEIGHTS = np.array([weight_A, weight_B])
WEIGHTS

array([ 0.33712572, -0.66287428])

In [19]:
def get_portfolio_avg_return(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_avg_return = mean.loc[long_sid]
    short_avg_return = mean.loc[short_sid]

    returns = np.array([long_avg_return, short_avg_return])

    portfolio_avg_returns = np.dot(WEIGHTS, returns)

    return (case, portfolio_avg_returns)

In [20]:
longshort_portfolio_returns = []

for case in longshort_cases:
    longshort_portfolio_returns.append(get_portfolio_avg_return(case))

In [21]:
longshort_portfolio_returns = sorted(longshort_portfolio_returns, key=lambda x: x[1], reverse=True)

In [22]:
# Top 5 return portfolios
longshort_portfolio_returns[:5]

[(('AAPL', 'NFLX'), 0.03479344759273418),
 (('GOOG', 'NFLX'), 0.02945917136184873),
 (('AMZN', 'NFLX'), 0.023163970517489238),
 (('AAPL', 'META'), 0.019719325886663308),
 (('META', 'NFLX'), 0.017332955044364065)]

In [23]:
# Bottom 5 return portfolios
longshort_portfolio_returns[-5:]

[(('META', 'GOOG'), -0.02158436934422214),
 (('AMZN', 'AAPL'), -0.02624188764021151),
 (('NFLX', 'GOOG'), -0.029250791638151266),
 (('META', 'AAPL'), -0.03207290311333668),
 (('NFLX', 'AAPL'), -0.03973932540726581)]

### 1-2 Portfolio's volatility

$$ PortfolioVariance = w_a^2 \sigma_a^2 + w_b^2 \sigma_b^2 + 2 \rho w_a w_b \sigma_a \sigma_b $$ 

In [24]:
def get_portfolio_variance(case: tuple) -> Tuple[tuple, float]:
    long_sid, short_sid = case
    
    long_std = std.loc[long_sid]
    short_std = std.loc[short_sid]

    corr_coef = corr_matrix_df.loc[long_sid, short_sid]

    stds = np.array([long_std, short_std])

    portfolio_variance = np.dot(WEIGHTS ** 2, stds ** 2) + 2 * corr_coef * np.prod(WEIGHTS) * np.prod(stds)

    return (case, portfolio_variance)

In [25]:
longshort_portfolio_vars = []

for case in longshort_cases:
    longshort_portfolio_vars.append(get_portfolio_variance(case))

In [26]:
longshort_portfolio_vars = sorted(longshort_portfolio_vars, key=lambda x: x[1], reverse=True)

In [27]:
# Top 5 variance portfolios
longshort_portfolio_vars[:5]

[(('AAPL', 'NFLX'), 6.317947732046371e-05),
 (('GOOG', 'NFLX'), 6.22448083809962e-05),
 (('META', 'NFLX'), 5.844037432633868e-05),
 (('AMZN', 'NFLX'), 5.5373417890564505e-05),
 (('NFLX', 'META'), 4.356765112181394e-05)]

In [28]:
# Botoom 5 variance portfolios
longshort_portfolio_vars[-5:]

[(('AMZN', 'AAPL'), 2.2246911366866702e-05),
 (('GOOG', 'AAPL'), 2.1775417246736545e-05),
 (('META', 'GOOG'), 1.7107775582243906e-05),
 (('AMZN', 'GOOG'), 1.6902541468644443e-05),
 (('AAPL', 'GOOG'), 1.6329774029774797e-05)]

### Sharpe ratio

Let's bring `r_f`(risk-free rate) from real world data.

https://ycharts.com/indicators/10_year_treasury_rate

The 10 Year Treasury Rate is the yield received for investing in a US government issued treasury security that has a maturity of 10 year. The 10 year treasury yield is included on the longer end of the yield curve. 

Many analysts will use the 10 year yield as the "risk free" rate when valuing the markets or an individual security.



**As of October 21, 2022:	4.21%**

In [33]:
r_f = 0.0421

In [34]:
port_returns_dict = dict(longshort_portfolio_returns)
port_vars_dict = dict(longshort_portfolio_vars)

In [35]:
longshort_portfolio_sharpe = []

for case in longshort_cases:
    ret = port_returns_dict[case]
    std = np.sqrt(port_vars_dict[case])

    sharpe = (ret-r_f) / std

    longshort_portfolio_sharpe.append((case, sharpe))

In [36]:
longshort_portfolio_sharpe = sorted(longshort_portfolio_sharpe, key=lambda x: x[1], reverse=True)

In [37]:
# Top 5 variance portfolios
longshort_portfolio_sharpe[:5]

[(('AAPL', 'NFLX'), -0.9192306345117941),
 (('GOOG', 'NFLX'), -1.6022267447936018),
 (('AMZN', 'NFLX'), -2.5447097608217453),
 (('META', 'NFLX'), -3.239796246583948),
 (('AAPL', 'META'), -3.533491516828713)]

### Answer

I'll choose `('AAPL', 'NFLX')` since it has the highest sharpe ratio

In [42]:
portfolio = ('AAPL', 'NFLX')

print(f'Portfolio: Long 1 {portfolio[0]} / Short 1 {portfolio[1]}')
print(f'Portfolio return: {port_returns_dict[portfolio]:.4f}')
print(f'Portfolio std: {np.sqrt(port_vars_dict[portfolio]):.4f}')
print(f'Portfolio sharpe: {longshort_portfolio_sharpe[0][1]:.4f}')

Portfolio: Long 1 AAPL / Short 1 NFLX
Portfolio return: 0.0348
Portfolio std: 0.0079
Portfolio sharpe: -0.9192


## Step 2

Answer the following questions:

Each Team Member/Portfolio Manager will answer the following questions about their
own portfolio:

### 2-1 Shorting

#### a. Can this portfolio be shorted? (Hint: Yes, but be sure to explain part b)

## Step 3